# Paso 12. Navier-Stokes, flujo de canal

En este paso utilizaremos las mismas ecuaciones del paso 11, con las siguientes modificaciones.
* Adición de una fuente de fluido $F$, esto con el fin de generar el flujo de canal.
* Cambio en las condiciones de frontera.

#### Navier-Stokes
$$
\frac{D\vec{V}}{Dt} = -\frac{1}{\rho} \nabla P + \nu \nabla^2 \vec{V} + F
$$
Donde $F(x,y) = (1, 0)$

#### Condiciones iniciales
$$
u(x,y) = v(x,y) = p(x,y) = 0 \,\,\,\,\forall (x, y) \in [0,2]\times[0,2]
$$

#### Condiciones de frontera

$$
u(0,y) = u(2,y)\\
v(0,y) = v(2,y)\\
p(0,y) = p(2,y)\\
$$

$$
u(x,0) = u(x,2) = 0\\
v(x,0) = v(x,2) = 0\\
$$

$$
\frac{\partial p(x,0)}{\partial y} = \frac{\partial p(x,2)}{\partial y} = 0\\
$$

#### Discretización

El único cambio que tenemos es para $u(i,j)_{n+1}$

$$
u(i,j)_{n+1} = u(i,j)_n - 
u(i,j)_n\frac{\Delta t}{\Delta x} \Big[u(i,j)_n-u(i-1,j)_n \Big] -
v(i,j)_n\frac{\Delta t}{\Delta y} \Big[u(i,j)_n-u(i,j-1)_n \Big] \\
-\frac{\Delta t}{2 \rho \Delta x} \Big[p(i+1,j)_n-p(i-1,j)_n\Big] \\
+\nu \Bigg[
\frac{\Delta t}{\Delta x^2} \big[ u(i+1,j)_n -2u(i,j)_n + u(i-1,j)_n \big] \\
+\frac{\Delta t}{\Delta y^2} \big[ u(i,j+1)_n -2u(i,j)_n + u(i,j-1)_n\big]
\Bigg] \\
+F \Delta t
$$

In [83]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

def PresureStabilization(rho, p, u, v):
    
    for i in range(50):
        p_n = p;
        pn_im1_j = np.roll(p,  1, axis=1)[1:-1,1:-1];
        pn_im1_j = np.roll(p,  1, axis=1)[1:-1,1:-1];
        pn_ip1_j = np.roll(p, -1, axis=1)[1:-1,1:-1];
        pn_i_jm1 = np.roll(p,  1, axis=0)[1:-1,1:-1];
        pn_i_jp1 = np.roll(p, -1, axis=0)[1:-1,1:-1];
        
        un_im1_j = np.roll(u,  1, axis=1)[1:-1,1:-1];
        un_ip1_j = np.roll(u, -1, axis=1)[1:-1,1:-1];
        un_i_jm1 = np.roll(u,  1, axis=0)[1:-1,1:-1];
        un_i_jp1 = np.roll(u, -1, axis=0)[1:-1,1:-1];

        vn_im1_j = np.roll(v,  1, axis=1)[1:-1,1:-1];
        vn_ip1_j = np.roll(v, -1, axis=1)[1:-1,1:-1];
        vn_i_jm1 = np.roll(v,  1, axis=0)[1:-1,1:-1];
        vn_i_jp1 = np.roll(v, -1, axis=0)[1:-1,1:-1];
        
        
        p[1:-1,1:-1] = ((pn_ip1_j+pn_im1_j)*dy**2 + (pn_i_jp1+pn_i_jm1)*dx**2) / \
                    (2*(dx**2+dy**2)) -\
                    (rho*(dx**2)*(dy**2))/ (2*(dx**2+dy**2)) * \
                    ( 1/dt * ((un_ip1_j-un_im1_j)/(2*dx) + (vn_i_jp1-vn_i_jm1)/(2*dy)) -\
                      ((un_ip1_j-un_im1_j)/(2*dx))**2 -\
                      2*((un_i_jp1-un_i_jm1)/(2*dy)*(vn_ip1_j-vn_im1_j)/(2*dx)) -\
                      ((vn_i_jp1-vn_i_jm1)/(2*dy))**2 \
                    );
        
        # Periodic Profile
        helper = p[1:-1,0].copy();
        p[1:-1,0] = p[1:-1,-1];
        p[1:-1,-1] = helper; 
                
        # Boundary Conditions
        p[-1,:] = p[-2,:] # dp/dy = 0 at y = 2
        p[0,:]  = p[1,:]  # dp/dy = 0 at y = 0
        
    return p;

def update(i, profile):
    #store the actual state of the arrays
    u_n = u;
    v_n = v;    
    p_n = p;

    p_n = PresureStabilization(rho, p, u, v);
    
    #update arrays, just for readability create dummy variables
    pn_im1_j = np.roll(p,  1, axis=1)[1:-1,1:-1];
    pn_ip1_j = np.roll(p, -1, axis=1)[1:-1,1:-1];
    pn_i_jm1 = np.roll(p,  1, axis=0)[1:-1,1:-1];
    pn_i_jp1 = np.roll(p, -1, axis=0)[1:-1,1:-1];
        
    un_i_j   = u_n[1:-1,1:-1];
    un_im1_j = np.roll(u,  1, axis=1)[1:-1,1:-1];
    un_ip1_j = np.roll(u, -1, axis=1)[1:-1,1:-1];
    un_i_jm1 = np.roll(u,  1, axis=0)[1:-1,1:-1];
    un_i_jp1 = np.roll(u, -1, axis=0)[1:-1,1:-1];
    
    vn_i_j   = v_n[1:-1,1:-1];
    vn_im1_j = np.roll(v,  1, axis=1)[1:-1,1:-1];
    vn_ip1_j = np.roll(v, -1, axis=1)[1:-1,1:-1];
    vn_i_jm1 = np.roll(v,  1, axis=0)[1:-1,1:-1];
    vn_i_jp1 = np.roll(v, -1, axis=0)[1:-1,1:-1];
    
    u[1:-1,1:-1] = un_i_j - \
                (un_i_j*(dt/dx)) * (un_i_j - un_im1_j) - \
                (vn_i_j*(dt/dy)) * (un_i_j - un_i_jm1) - \
                dt/(2*rho*dx)*( pn_ip1_j - pn_im1_j ) + \
                nu*(dt/dx**2)*( un_ip1_j - 2*un_i_j + un_im1_j ) + \
                nu*(dt/dy**2)*( un_i_jp1 - 2*un_i_j + un_i_jm1 ) + \
                F*dt;
        
    v[1:-1,1:-1] = vn_i_j - \
                (un_i_j*(dt/dx)) * (vn_i_j - vn_im1_j) - \
                (vn_i_j*(dt/dy)) * (vn_i_j - vn_i_jm1) - \
                dt/(2*rho*dy)*( pn_i_jp1 - pn_i_jm1 ) + \
                nu*(dt/dx**2)*( vn_ip1_j - 2*vn_i_j + vn_im1_j ) + \
                nu*(dt/dy**2)*( vn_i_jp1 - 2*vn_i_j + vn_i_jm1 );
                
    # Periodic profile
    helper = u[1:-1,0].copy();
    u[1:-1,0] = u[1:-1,-1];
    u[1:-1,-1] = helper; 
    
    helper = v[1:-1,0].copy();
    v[1:-1,0] = v[1:-1,-1];
    v[1:-1,-1] = helper; 
    
    # Boundary Conditions
    u[0,:]  = 0
    u[-1,:] = 0
    
    v[0,:]  = 0
    v[-1,:] = 0
    
    ax.clear();

    #profile = ax.contourf(X, Y, u, alpha=1.0);
    
    try:
        ax.contour(X, Y, p, alpha=0.25);
    except:
        pass;
    
    ax.quiver(X[::3,::3],Y[::3,::3],u[::3,::3],v[::3,::3]);
    
    return profile,


# How much we want to advance in time?
# at 30 fps if we want simulate 10 seconds => 30*10
t_seconds = 30 * 10;

partitions = 51;

data_x = np.linspace(0.0, 2.0, num=partitions, retstep=True);
x  = data_x[0]
dx = data_x[1];

data_y = np.linspace(0.0, 2.0, num=partitions, retstep=True);
y  = data_x[0]
dy = data_x[1];

X, Y = np.meshgrid(x, y)

# density
rho = 1.0;
# viscocity
nu = 0.1
# The time step size 
dt = 0.001;

# Fluid source
F = 1.0;

#initial conditions
u = np.zeros((partitions, partitions));
v = np.zeros((partitions, partitions));
p = np.zeros((partitions, partitions));
b = np.zeros((partitions, partitions));

# Create a figure and a 3D Axes
fig = plt.figure(figsize=(7,7), dpi=100)
ax = plt.axes(xlim=(0, 2), ylim=(0, 2)) 

profile = ax.contourf(X, Y, p, alpha=0.5)

# Animate
anim = animation.FuncAnimation(fig, update, fargs=(profile,), frames=t_seconds, blit=False)
anim.save('Navier-Stokes_Channel-Flow.mp4', fps=30, writer="ffmpeg", codec="libx264");
plt.close(fig);

In [84]:
# You must have configured ffmpeg in your machine to run the code below
from IPython.display import HTML
video = open("Navier-Stokes_Channel-Flow.mp4", "rb").read()
video_encoded = video.encode("base64")
video_tag = '<video controls alt="test" src="data:video/x-m4v;base64,{0}">'.format(video_encoded)
HTML(video_tag)